In [1]:
import sqlite3
import geopandas as gp

In [2]:
conn = sqlite3.connect('taxi_db_s.db')
c = conn.cursor()

In [3]:
c.execute("create table if not exists Date_time as select rowid as ID, substr(tpep_pickup_datetime,1,10) as PU_Date, substr(tpep_pickup_datetime,12,2) as PU_hour,substr(tpep_pickup_datetime,15,2) as PU_minute, substr(tpep_pickup_datetime,21,2) as PU_AM_PM, substr(tpep_dropoff_datetime,1,10) as DO_Date, substr(tpep_dropoff_datetime,12,2) as DO_hour, substr(tpep_dropoff_datetime,15,2) as DO_minute, substr(tpep_dropoff_datetime,21,2) as DO_AM_PM from taxi_details")

In [4]:
c.execute("create table if not exists taxiDetails as select rowid as ID,VendorID, passenger_count, trip_distance, RatecodeID, store_and_fwd_flag,PULocationID, DOLocationID, payment_type, fare_amount, extra, mta_tax, tip_amount, tolls_amount, improvement_surcharge, total_amount from taxi_details")

In [6]:
shp_file = gp.read_file('taxi_zones.shp')

In [9]:
import pandas as pd
import numpy as np
import urllib.request
import zipfile
import random
import itertools
import math
import shapefile
from shapely.geometry import Polygon
from descartes.patch import PolygonPatch
import matplotlib as mpl
import matplotlib.pyplot as plt
import sqlite3 as db

plt.style.use('ggplot')

def get_lat_lon(sf):
    content = []
    for sr in sf.shapeRecords():
        shape = sr.shape
        rec = sr.record
        loc_id = rec[shp_dic['LocationID']]
        
        x = (shape.bbox[0]+shape.bbox[2])/2
        y = (shape.bbox[1]+shape.bbox[3])/2
        
        content.append((loc_id, x, y))
    return pd.DataFrame(content, columns=["LocationID", "longitude", "latitude"])

sf = shapefile.Reader("taxi_zones.shp")
fields_name = [field[0] for field in sf.fields[1:]]
shp_dic = dict(zip(fields_name, list(range(len(fields_name)))))
attributes = sf.records()
shp_attr = [dict(zip(fields_name, attr)) for attr in attributes]
df_loc = pd.DataFrame(shp_attr).join(get_lat_lon(sf).set_index("LocationID"), on="LocationID")

df_loc.to_csv("outLatLong.csv")
df_latlon = pd.read_csv("outLatLong.csv")
df_latlon.head()

,Unnamed: 0,LocationID,OBJECTID,Shape_Area,Shape_Leng,borough,zone,longitude,latitude
0,0,1,1,0.000782,0.116357,EWR,Newark Airport,9.366817e+05,190522.130278
1,1,2,2,0.004866,0.433470,Queens,Jamaica Bay,1.033536e+06,161853.982300
2,2,3,3,0.000314,0.084341,Bronx,Allerton/Pelham Gardens,1.027136e+06,254730.010849
3,3,4,4,0.000112,0.043567,Manhattan,Alphabet City,9.904240e+05,203100.040432
4,4,5,5,0.000498,0.092146,Staten Island,Arden Heights,9.321332e+05,139954.541936
